##### https://machinelearningmastery.com/evaluate-machine-learning-algorithms-for-human-activity-recognition/

## Classifiers for activity recognition

In [155]:
# spot check on raw data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import dstack
from pandas import read_csv
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [158]:
def normalise(df):
    return (df - df.min()) / (df.max() - df.min())

In [179]:
df = pd.read_csv('acc_data (right_wrist).csv')
#df.dropna()
df = df.drop([0])
#df = normalise(df)
print(df)
df.info

     PacketCounter     Acc_X     Acc_Y     Acc_Z Unnamed: 11
1                1 -8.874124 -0.679627  2.768368            
2                2 -8.873473 -0.613973  2.761706            
3                3 -8.834931 -0.518206  2.797576            
4                4 -8.921975 -0.508252  2.836315            
5                5 -8.999269 -0.533706  2.816627            
..             ...       ...       ...       ...         ...
398            398 -8.989089 -2.510464  2.123948            
399            399 -9.029768 -2.650938  2.112896            
400            400 -9.044131 -2.731521  2.112670            
401            401 -8.995914 -2.801886  2.085057            
402            402 -8.966281 -2.829942  2.044567            

[402 rows x 5 columns]


<bound method DataFrame.info of      PacketCounter     Acc_X     Acc_Y     Acc_Z Unnamed: 11
1                1 -8.874124 -0.679627  2.768368            
2                2 -8.873473 -0.613973  2.761706            
3                3 -8.834931 -0.518206  2.797576            
4                4 -8.921975 -0.508252  2.836315            
5                5 -8.999269 -0.533706  2.816627            
..             ...       ...       ...       ...         ...
398            398 -8.989089 -2.510464  2.123948            
399            399 -9.029768 -2.650938  2.112896            
400            400 -9.044131 -2.731521  2.112670            
401            401 -8.995914 -2.801886  2.085057            
402            402 -8.966281 -2.829942  2.044567            

[402 rows x 5 columns]>

In [180]:
#df = pd.read_csv('acc_data (right_wrist).csv')
#df["Acc_X"] = df['Acc_X'].replace(' ', '').astype(float)
#print(df.values)
#df = df.replace(r'^\s*$', np.nan, regex=True)
#print(df.values)

#df_X = df[['Acc_X']].astype({'Acc_X': float})
#print(df_X)
#df_X.info()

#df_Y = df[['Acc_Y']].astype({'Acc_Y': float})
#print(df_Y)
##df_Y.info()

#df_Z = df[['Acc_Z']].astype({'Acc_Z': float})
#print(df_Z)
#df_Z.info()

#df = np.column_stack((np.array(df_X), np.array(df_Y), np.array(df_Z)

X = np.column_stack((np.array(df['Acc_X']), np.array(df['Acc_Y']), np.array(df['Acc_Z'])))
y = np.array(df['PacketCounter']).reshape(-1, 1)

#loaded = list()
#loaded.append(df)
#loaded = dstack(loaded)

#X = loaded
#y = df

print(X.shape, y.shape)

(402, 3) (402, 1)


In [164]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [165]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
# IMPORTANT: the scaler must be fitted with the training data only. Transfromation of the test and prediction sets
# Must be done with the scaler fitted with the training data.
# This scaler scales one-hot variables
trainX = sc_X.fit_transform(trainX)
testX = sc_X.transform(testX)


In [166]:
print(trainX.shape, trainy.shape)

(321, 3) (321, 1)


In [167]:
print(testX.shape, testy.shape)

(81, 3) (81, 1)


In [168]:
# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
	# fit the model
	model.fit(trainX, trainy.ravel())
	# make predictions
	yhat = model.predict(testX)
	# evaluate predictions
	accuracy = accuracy_score(testy, yhat)
	return accuracy * 100.0

In [169]:
# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
	results = dict()
	for name, model in models.items():
		# evaluate the model
		results[name] = evaluate_model(trainX, trainy, testX, testy, model)
		# show processyh
		print('>%s: %.3f' % (name, results[name]))
	return results

In [170]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
	# nonlinear models
	models['knn'] = KNeighborsClassifier(n_neighbors=7)
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	# ensemble models
	models['bag'] = BaggingClassifier(n_estimators=100)
	models['rf'] = RandomForestClassifier(n_estimators=100)
	models['et'] = ExtraTreesClassifier(n_estimators=100)
	models['gbm'] = GradientBoostingClassifier(n_estimators=100)
	print('Defined %d models' % len(models))
	return models

In [171]:
# get model list
models = define_models()

Defined 8 models


In [172]:
# evaluate models
results = evaluate_models(trainX, trainy, testX, testy, models)

>knn: 0.000
>cart: 0.000
>svm: 0.000
>bayes: 0.000
>bag: 0.000
>rf: 0.000
>et: 0.000
>gbm: 0.000


In [173]:
# print and plot the results
def summarize_results(results, maximize=True):
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k,v) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	print()
	for name, score in mean_scores:
		print('Name=%s, Score=%.3f' % (name, score))


In [174]:
# summarize results
summarize_results(results)


Name=gbm, Score=0.000
Name=et, Score=0.000
Name=rf, Score=0.000
Name=bag, Score=0.000
Name=bayes, Score=0.000
Name=svm, Score=0.000
Name=cart, Score=0.000
Name=knn, Score=0.000


#### Train

In [53]:
model = SVC()
model.fit(trainX, trainy.ravel())

SVC()

In [56]:
yhat = model.predict(testX)
accuracy = accuracy_score(testy, yhat)
print(accuracy)

0.0


In [307]:
#trainX = trainX.reshape((trainX.shape[0], trainX.shape[1] * trainX.shape[2]))

#testX = testX.reshape((testX.shape[0], testX.shape[1] * testX.shape[2]))


In [308]:
# flatten y
#trainy, testy = trainy[:,0], testy[:,0]
#print(trainX.shape, trainy.shape, testX.shape, testy.shape)

In [309]:
models = KNeighborsClassifier(n_neighbors=7)
models.fit(trainX, trainy)

ValueError: could not convert string to float: 

In [301]:
model = SVC()
model.fit(trainX, trainy)

ValueError: could not convert string to float: 

In [234]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
	# nonlinear models
	models['knn'] = KNeighborsClassifier(n_neighbors=7)
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	# ensemble models
	models['bag'] = BaggingClassifier(n_estimators=100)
	models['rf'] = RandomForestClassifier(n_estimators=100)
	models['et'] = ExtraTreesClassifier(n_estimators=100)
	models['gbm'] = GradientBoostingClassifier(n_estimators=100)
	print('Defined %d models' % len(models))
	return models

In [235]:
# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
	# fit the model
	model.fit(trainX, trainy)
	# make predictions
	yhat = model.predict(testX)
	# evaluate predictions
	accuracy = accuracy_score(testy, yhat)
	return accuracy * 100.0

In [236]:
# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
	results = dict()
	for name, model in models.items():
		# evaluate the model
		results[name] = evaluate_model(trainX, trainy, testX, testy, model)
		# show process
		print('>%s: %.3f' % (name, results[name]))
	return results

In [237]:
# print and plot the results
def summarize_results(results, maximize=True):
	# create a list of (name, mean(scores)) tuples
	mean_scores = [(k,v) for k,v in results.items()]
	# sort tuples by mean score
	mean_scores = sorted(mean_scores, key=lambda x: x[1])
	# reverse for descending order (e.g. for accuracy)
	if maximize:
		mean_scores = list(reversed(mean_scores))
	print()
	for name, score in mean_scores:
		print('Name=%s, Score=%.3f' % (name, score))

In [238]:
# load dataset
#trainX, trainy, testX, testy = load_dataset()
# get model list
models = define_models()
# evaluate models
results = evaluate_models(trainX, trainy, testX, testy, models)
# summarize results
summarize_results(results)

Defined 8 models


ValueError: could not convert string to float: 